In [ ]:
import pandas as pd

def merge_all_data(user_health_data_file, supplement_usage_file, experiments_file, user_profiles_file):
    """
    Combines and cleans datasets for analysis.
    """
    # Load data
    health_data = pd.read_csv(user_health_data_file)
    usage_data = pd.read_csv(supplement_usage_file)
    experiments = pd.read_csv(experiments_file)
    profiles = pd.read_csv(user_profiles_file)

    # Combine supplement usage with experiments
    # Left join is used to keep all records from supplement_usage, even if there's no experiment metadata available.
    usage_data = usage_data.merge(experiments, on='experiment_id', how='left').rename(columns={'name': 'experiment_name'})

    # Merge health data with usage data
    # Outer join is used to retain all health data and supplement usage records, even if a user has data in only one dataset.
    combined_data = health_data.merge(usage_data, on=['user_id', 'date'], how='outer')

    # Add user profiles
    # Left join is used to preserve all combined data while adding demographic details for users. Missing profiles are acceptable.
    combined_data = combined_data.merge(profiles, on='user_id', how='left')

    # Format date
    combined_data['date'] = pd.to_datetime(combined_data['date'], errors='coerce')

    # Age group classification
    def age_group(age):
        if pd.isnull(age):
            return 'Unknown'
        if age < 18:
            return 'Under 18'
        if age <= 25:
            return '18-25'
        if age <= 35:
            return '26-35'
        if age <= 45:
            return '36-45'
        if age <= 55:
            return '46-55'
        if age <= 65:
            return '56-65'
        return 'Over 65'

    combined_data['user_age_group'] = combined_data['age'].apply(age_group)

    # Handle missing supplement data
    combined_data['supplement_name'] = combined_data['supplement_name'].fillna('No intake')

    # Convert dosage to grams
    combined_data['dosage_grams'] = combined_data.apply(
        lambda row: row['dosage'] / 1000 if row.get('dosage_unit') == 'mg' else row['dosage'], axis=1
    )

    # Drop unused columns
    combined_data.drop(columns=['dosage', 'dosage_unit'], inplace=True)

    # Parse sleep hours as float
    combined_data['sleep_hours'] = combined_data['sleep_hours'].str.extract(r'(\d+\.?\d*)').astype(float)

    # Convert is_placebo to boolean
    combined_data['is_placebo'] = combined_data['is_placebo'].astype('boolean')

    # Select final columns
    final_columns = [
        'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
        'supplement_name', 'dosage_grams', 'is_placebo',
        'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'
    ]
    return combined_data[final_columns]

# Run the function
final_data = merge_all_data(
    'user_health_data.csv',
    'supplement_usage.csv',
    'experiments.csv',
    'user_profiles.csv'
)

# Display the result
print(final_data)
